In [1]:
from cytoolz import filter, map, partial, reduce
from cytoolz.dicttoolz import assoc,dissoc
from cytoolz.functoolz import thread_first
from cytoolz.itertoolz import join, mapcat
import json
import re

In [14]:
def times_two(number):
    return number * 2

In [15]:
def plus_one(number):
    return times_two(number)

In [16]:
number_list = [1, 2, 3, 4]

In [18]:
list(map(plus_one, number_list))

In [2]:
# def remove_key(record, key):
#     return record.pop(key, None)

# res = filter(partial(remove_key, dict_result), arrays)

In [3]:
# array_keys: list = [k for k, v in record.items() if (type(v) == list)]
# arrays: dict = {k: v for k, v in flat_record.items() if k in vector_keys}
# arrays: dict = reduce(dissoc, set(flats), record)
# flats: dict = {k: v for k, v in flat_record.items() if k not in vector_keys}

In [4]:
def put_prefix(previous_key):
    if (previous_key == None):
        prefix = ''
    else:
        prefix = f'{previous_key}_'
    return prefix

In [5]:
def replace_char(key_name, expression):
    return re.sub(expression, '', key_name)


def clean_key(key_name):
    if (key_name == '_id_$oid'):
        new_key = thread_first(
            key_name, (replace_char, '^\\_'), (replace_char, '\\$')
        )
    elif ('$' in key_name):
        new_key = thread_first(
            key_name, (replace_char, '\\b\\$\\w+'), (replace_char, '_$')
        )
    else:
        new_key = key_name
    return new_key

In [6]:
def unnest_data(prefix, key_value):
    key = f'{put_prefix(prefix)}{key_value[0]}'
    if (key_value[1] == None):
        pass
    elif (type(key_value[1]) == dict):
        yield from mapcat(partial(unnest_data, key), key_value[1].items())
    else:
        new_key = clean_key(key)
        value = key_value[1]
        yield { new_key: value }

In [7]:
def is_array(record):
    if (type(record[1]) == list):
        return record

In [8]:
def process_record(json_record):
    pairs = mapcat(partial(unnest_data, None), json_record.items())
    record = reduce(lambda x, y: x | y, pairs)
    arrays = dict(filter(is_array, record.items()))

    if arrays.keys():
        flats = reduce(dissoc, arrays.keys(), record)
        exploded = [flats | {k: i} for k in arrays.keys() for i in arrays[k]]
        yield from mapcat(process_record, exploded)
    else:
        yield record


def call_processing(raw_record):
    json_record = json.loads(raw_record)
    return process_record(json_record)

In [9]:
with open('example.json', 'r') as f:
    raw_file = f.readlines()

In [10]:
pairs = mapcat(partial(unnest_data, None), json.loads(raw_file[0]).items())

In [11]:
record = reduce(lambda x, y: x | y, pairs)

In [12]:
arrays = dict(filter(is_array, record.items()))

In [13]:
flats = reduce(dissoc, arrays.keys(), record)

In [27]:
for i in arrays.values():
    print(i)

[{'id': 123, 'name': 'metamizole'}, {'id': 456, 'name': 'aspirin'}, {'id': 789, 'name': 'paracetamol'}, {'id': 101, 'name': 'ibuprofen'}]
[{'name': 'pneumonia'}, {'name': 'fever'}, {'name': 'nausea'}]


In [18]:
exploded = [flats | {k: v} for k in arrays.keys() for v in arrays[k]]

In [19]:
exploded

[{'id_oid': 'abcd1234EFGH',
  'hospital_id': 42,
  'document_discharge_date': '2022-01-02',
  'event_date': '2022-01-02',
  'drugs': {'id': 123, 'name': 'metamizole'}},
 {'id_oid': 'abcd1234EFGH',
  'hospital_id': 42,
  'document_discharge_date': '2022-01-02',
  'event_date': '2022-01-02',
  'drugs': {'id': 456, 'name': 'aspirin'}},
 {'id_oid': 'abcd1234EFGH',
  'hospital_id': 42,
  'document_discharge_date': '2022-01-02',
  'event_date': '2022-01-02',
  'drugs': {'id': 789, 'name': 'paracetamol'}},
 {'id_oid': 'abcd1234EFGH',
  'hospital_id': 42,
  'document_discharge_date': '2022-01-02',
  'event_date': '2022-01-02',
  'drugs': {'id': 101, 'name': 'ibuprofen'}},
 {'id_oid': 'abcd1234EFGH',
  'hospital_id': 42,
  'document_discharge_date': '2022-01-02',
  'event_date': '2022-01-02',
  'conditions': {'name': 'pneumonia'}},
 {'id_oid': 'abcd1234EFGH',
  'hospital_id': 42,
  'document_discharge_date': '2022-01-02',
  'event_date': '2022-01-02',
  'conditions': {'name': 'fever'}},
 {'id_o

In [11]:
result_iter = [call_processing(x) for x in raw_file]

In [12]:
result_list = [y for x in result_iter for y in x]

In [ ]:
with open('result.json', 'w') as f:
    for element in result_list:
        f.write(f'{json.dumps(element)}\n')

In [1]:
import polars as pl

In [3]:
df = pl.read_parquet('../json_steamroller/c71c319904bd4195ac60615a0e011261.parquet')

In [ ]:
df.filter(pl.col('atendimento_id').str.contains('48657422123'))

In [8]:
dfnew = df.select(pl.col('*').cast(str))

In [10]:
dfnew.write_parquet('result.parquet')